### Feature 6 - Usage of Gas Tokens

In [20]:
from web3 import Web3
from tqdm import tqdm
import pandas as pd


In [21]:
web3 = Web3(Web3.HTTPProvider("https://intensive-sly-mountain.quiknode.pro/a3f5256d7f2af6541d483cce3f1d49c94c01879e/"))
print(web3.is_connected())

True


In [33]:
web3.eth.get_transaction('0x10504c0666a22466911d038d2b1c895b7ea5ec05cef107e46d3cae0121cfcda7')


AttributeDict({'blockHash': HexBytes('0x58a502dd0f64ab442535f89d9118e743e2fec44b435bf41a9e0fda92ec92b0f2'),
 'blockNumber': 10912913,
 'from': '0x30b8235f492265A734347C0bF36E2FFcAD887be2',
 'gas': 450000,
 'gasPrice': 1494720129875,
 'hash': HexBytes('0x10504c0666a22466911d038d2b1c895b7ea5ec05cef107e46d3cae0121cfcda7'),
 'input': HexBytes('0x94264cbc0000000000000000000000000000000000000000000000015af1d78b58c40000000000000000000000000000000000000000000000000008c95411b03241f9e9000000000000000000000000000000000000000000000000000000005f6a1906000000000000000000000000000000000000000000000000000000005f6a135e'),
 'nonce': 6182,
 'to': '0x3F499Fe484a61Af03b01ea5C81Eab14F67C11A5A',
 'transactionIndex': 0,
 'value': 0,
 'type': 0,
 'chainId': 1,
 'v': 38,
 'r': HexBytes('0xb706c2b6b16f61f9c720e7a88ebbd2ab21136bde7adf4fa5443664435b5afae4'),
 's': HexBytes('0x0e930ba73fefae8b7d9df560de61e3e16cd644a25286fdbfb5410b8d1c900996')})

In [78]:
web3.eth.get_transaction('0x8abbef4de9993df03fcba73f80d0d245939836fbf75265d80df84b852ce39453')

AttributeDict({'blockHash': HexBytes('0xd75c6d969fc00d9d23ce400f5d0dd1933062e346af3649bb2c92dc10d55660cc'),
 'blockNumber': 9675068,
 'from': '0x99746fbd4E138D909Ba07b4232173a677D11a5bc',
 'gas': 400000,
 'gasPrice': 49280111111,
 'hash': HexBytes('0x8abbef4de9993df03fcba73f80d0d245939836fbf75265d80df84b852ce39453'),
 'input': HexBytes('0x7019ef400000000000000000000000000000000000000000000002574959d59edb89b0f10000000000000000000000000000000000000000000002b5e3af16b1e66def590000000000000000000000000000000000000000000000056bc75e2d63100000000000000000000000000000000000000000000000000000000000005e6def590000000000000000000000000000000000000000000000008f362847a5c613c8000000000000000000000000000000000000000000000000000000000001b207'),
 'nonce': 73,
 'to': '0x570cf89349c712744c13b5b1749F00F2782AD88a',
 'transactionIndex': 14,
 'value': 0,
 'type': 0,
 'chainId': 1,
 'v': 38,
 'r': HexBytes('0x44f6a284873ba519a948adc4b2b94dae426a0472b930ab559fdd7a91901cd2ba'),
 's': HexBytes('0x16e220492cce24b98

#### Naive approach with web3 - Loop through AccessList

In [53]:
gas_token_addresses = {"0x0000000000b3F879cb30FE243b4Dfee438691c04": "GST2",
                       "0x88d60255F917e3eb94eaE199d827DAd837fac4cB": "GST1",
                       "0x0000000000004946c0e9F43F4Dee607b0eF1fA1c": "CHI"}

In [6]:
block_number = 19587005
block = web3.eth.get_block(block_number, full_transactions=True)

for transaction in block.transactions:  
    
    if (transaction["from"] == "0x678111a6cA5749f1744b5E080A855CEC8d631E20"):
        
        if "accessList" not in transaction:
            print("AccessList not in Transaction")
        
        for element in transaction["accessList"]:
            if element["address"] in gas_token_addresses.values():
                print(element["address"])


0x0000000000b3F879cb30FE243b4Dfee438691c04


In [7]:
def is_transaction_using_gas_token(block_number, address, web3):
    block = web3.eth.get_block(block_number, full_transactions=True)
    
    for transaction in block.transactions:   
        
        if not transaction["from"] == address:
            continue
    
        if "accessList" not in transaction:
            #print(transaction)
            continue
            
        for element in transaction["accessList"]:
            if element["address"] in gas_token_addresses.values():
                print(element["address"])
                return True
    return False

In [22]:
df_insertion = pd.read_csv ('../data/insertion_attacks.csv', delimiter=',')


In [9]:
for index, entry in tqdm(df_insertion.sample(1000).iterrows()):
        block_nr = entry["Block Number"]
        address = entry["First Attacker"]
        
        is_transaction_using_gas_token(block_nr, address, web3)

17it [00:06,  2.61it/s]


KeyboardInterrupt: 

#### Approach using web3 API - Recursive scanning of AttributeDict

In [10]:
import web3.datastructures as wd

def check_value_of_interest(data, transactions_with_value_of_interest, hash):
    
    if isinstance(data, wd.AttributeDict):
        for key, value in data.items():
            if value in gas_token_addresses.values():
                gas_token = list(filter(lambda x: gas_token_addresses[x] == value, gas_token_addresses))[0]
                transactions_with_value_of_interest.append((data, gas_token))
                print(f"Gas Token used in {data}: {gas_token} -> {value}")
                break  
            check_value_of_interest(value, transactions_with_value_of_interest, hash)
    
    elif isinstance(data, list):
        for item in data:
            check_value_of_interest(item, transactions_with_value_of_interest, hash)

# List to store transactions with the value of interest
transactions_with_value_of_interest = []

transactions = web3.eth.get_transaction('0xdcdc49f7bf2d1c97134df53de8090aef064b4611c17490020ace706cffecd35e')

# Call the recursive function for each AttributeDict object
for transaction_data in [transactions]:  # assuming you have two AttributeDict objects
    check_value_of_interest(transaction_data, transactions_with_value_of_interest, '0xdcdc49f7bf2d1c97134df53de8090aef064b4611c17490020ace706cffecd35e')

# Print the transactions with the value of interest
for transaction in transactions_with_value_of_interest:
    print(transaction)


Gas Token used in AttributeDict({'address': '0x0000000000b3F879cb30FE243b4Dfee438691c04', 'storageKeys': [HexBytes('0x5f7ca87abce7a5bad34de37cf054bbbda2853a1b5cddd3c3375ac67b5f088ee2'), HexBytes('0xf99c4b6cf8f9d6ad9f5a20955aa154326a813912ac137323e5300fa95c1fc5e5')]}): GST2 -> 0x0000000000b3F879cb30FE243b4Dfee438691c04
(AttributeDict({'address': '0x0000000000b3F879cb30FE243b4Dfee438691c04', 'storageKeys': [HexBytes('0x5f7ca87abce7a5bad34de37cf054bbbda2853a1b5cddd3c3375ac67b5f088ee2'), HexBytes('0xf99c4b6cf8f9d6ad9f5a20955aa154326a813912ac137323e5300fa95c1fc5e5')]}), 'GST2')


In [23]:

def is_transaction_using_gas_token(block_number, address, web3):
    block = web3.eth.get_block(block_number, full_transactions=True)
    
    transactions_with_value_of_interest = []
    
    for transaction in block.transactions:  
        
        if not (transaction["from"] == address or transaction["to"] == address):
            continue
        else:
            hash = transaction["hash"].hex()
            print(f"Transaction: {hash}\n")
            print(transaction)
            print("\n")

        
        check_value_of_interest(transaction, transactions_with_value_of_interest, transaction["hash"])
        

In [24]:
for index, entry in tqdm(df_insertion.sample(10).iterrows()):
        block_nr = entry["Block Number"]
        address = entry["First Attacker"]
        
        is_transaction_using_gas_token(block_nr, address, web3)

2it [00:00,  4.60it/s]

Transaction: 0x3e02aefb1e45b3c3dfc5f03a26680ac755fc7f656396b09a9a088e67cbafde5b

AttributeDict({'blockHash': HexBytes('0x84a1a6b43d98b3f96af07c557bc83d190fdeb1c501db5ee27c3384477a7a26ff'), 'blockNumber': 11166327, 'from': '0x4F69c5B694D5A14a0a595703175c478Ec6B2a2Fe', 'gas': 250000, 'gasPrice': 76483903996, 'hash': HexBytes('0x3e02aefb1e45b3c3dfc5f03a26680ac755fc7f656396b09a9a088e67cbafde5b'), 'input': HexBytes('0x000000010000000000000381846044db37951b520000000000000382b2c5e6686f581b5200000000000000000000000087febfb3ac5791034fd5ef1a615e9d9627c2665d'), 'nonce': 15140, 'to': '0x00bF034ceC51975FaeCe51b285724301436B21C4', 'transactionIndex': 21, 'value': 0, 'type': 0, 'chainId': 1, 'v': 38, 'r': HexBytes('0xaaf074ce20c4e315d67bd79c853bbd0e089d5a2843eb558bc9daf9051ee9ab7a'), 's': HexBytes('0x2dd6eb453128239b7dc6d12f2801035cebcfd6bf9354cbe463fdea5a864ccc29')})


Transaction: 0x63f9ebdff92e48af0df513363d3d142b8ed1b8ccc4ba24a0bfe38689a6cb2946

AttributeDict({'blockHash': HexBytes('0xe913efbc009

3it [00:00,  4.37it/s]

Transaction: 0xf8928fb2dafd5a484134ca6e9e23f2a39e7f5add92785e27cd76c5f2a36a9045

AttributeDict({'blockHash': HexBytes('0x384648d1e625e751358626962d617f2e2d588f65497af8e5cfba6458b9f2b578'), 'blockNumber': 11246677, 'from': '0x0Dd2811fF0F7C58c505957B5CaE0833ba6826DB3', 'gas': 400000, 'gasPrice': 112500000000, 'hash': HexBytes('0xf8928fb2dafd5a484134ca6e9e23f2a39e7f5add92785e27cd76c5f2a36a9045'), 'input': HexBytes('0x7a4e5cd200000000000000000000000095a4492f028aa1fd432ea71146b433e7b4446611000000000000000000000000000000000000000000000000908119109ef110000000000000000000000000000000000000000000000001a50e0e2bbfb53000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000005fadf3d4'), 'nonce': 2085, 'to': '0xfE201551e800871f6ce0FaF141107F298440F570', 'transactionIndex': 4, 'value': 0, 'type': 0, 'v': 27, 'r': HexBytes('0xbc03b0cc7ab229927f4024c42883610237171801ab76f204d3454b0d2fea3848'), 's': HexBytes('0x4f39bd0f154a5c70aba2df3a90

4it [00:00,  4.30it/s]

...

Transaction: 0x4d62d87ab044304588a6104fe6723b8b0ac0bb952aa4e0da4bd85167e3649b5a

AttributeDict({'blockHash': HexBytes('0xae7e280f5549e2724d04db50e9aa598d87cdeebc10a6ad1f48abba7b6b1a3362'), 'blockNumber': 11105864, 'from': '0x94D0060BE48a80b6f66F9069B8D82537708Fce4B', 'gas': 400000, 'gasPrice': 143000000000, 'hash': HexBytes('0x4d62d87ab044304588a6104fe6723b8b0ac0bb952aa4e0da4bd85167e3649b5a'), 'input': HexBytes('0x1d8704fa00000000000000000000000000000000000000000000000010a741a462780000000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000a0000000000000000000000000000000000000000000000000000000006b7b02af0000000000000000000000007a250d5630b4cf539739df2c5dacb4c659f2488d0000000000000000000000000000000000000000000000000000000000000002000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2000000000000000000000000aaaebe6fe48e54f431b0c390cfaf0b017d09d42d'), 'nonce': 154, 'to': '0x24A5EC7650761642b128A8883a30f419e1B

9it [00:02,  2.82it/s]

Transaction: 0xace4bdd3b60a132178a85c5d9b565c655dd077e143c32161c524d7701eb4a615

AttributeDict({'blockHash': HexBytes('0xe97f0b40d1412ca3127fce15cbdabd0fa02c8dec95e07ff555d7d645827f0d9f'), 'blockNumber': 11061737, 'from': '0x3C28AFa9c4de3f245C550340B2205286723193ce', 'gas': 300000, 'gasPrice': 120000000001, 'hash': HexBytes('0xace4bdd3b60a132178a85c5d9b565c655dd077e143c32161c524d7701eb4a615'), 'input': HexBytes('0x7b0b42669d0cf148df35ee2f4b9b3c1ab349d1067eb643e97301f1625d849daf2991260012fea1389946e42071ae135100fe5fdb60463ea208b03c7f4efdc1cb6b95feab952c2dbeda348fe489e0206668ecf92dc2d59b224bb1b0888fa9319fc4e73ce1'), 'nonce': 1839, 'to': '0xAfE0e7De1FF45Bc31618B39dfE42dd9439eEBB32', 'transactionIndex': 2, 'value': 0, 'type': 0, 'v': 27, 'r': HexBytes('0xaf3393f3909e7f4386e23382f40e231e3133fc3f06783631f83ecd8a4de22f13'), 's': HexBytes('0x4ab2a5685ab5da4c598161b83e01e28ebb327ee1305d70f0ba8524dbb7181ea9')})


Transaction: 0x4682cb606eb07e74bf660bdabbeffa5e8abd38f960893ae245f9c635a8eca35d

At

10it [00:02,  3.41it/s]

Transaction: 0x75e5877932d355d056a8d2a3f25dc041953edc9b4e638661a407a72884d96ad0

AttributeDict({'blockHash': HexBytes('0x2929e7a029fe3dfe6f67e2d9dceeb740265ba6298a625f723403aea373ee5e66'), 'blockNumber': 11265042, 'from': '0x7d3B97857B3200694dc161729605af6880Db9cD5', 'gas': 300000, 'gasPrice': 437424900000, 'hash': HexBytes('0x75e5877932d355d056a8d2a3f25dc041953edc9b4e638661a407a72884d96ad0'), 'input': HexBytes('0x79c7ee2f0000000000000000000000003f406840056b5afafdd9a3a865c48f792f7e99b400000000000000000000000066f0cbe6b95c024af85181ad2e87ae478e97fe830000000000000000000000000000000000000000000000003c0a89f277460000000000000000000000000000000000000000000000000002b9e6dbf9a2726000'), 'nonce': 11694, 'to': '0xFd52a4bD2289AEcCF8521F535Ec194B7e21cdC96', 'transactionIndex': 0, 'value': 0, 'type': 0, 'chainId': 1, 'v': 38, 'r': HexBytes('0x494dff2b84ef8d8c4ad0fba9d9e129431670cc7469d38072aa226a4bfa2909cd'), 's': HexBytes('0x0988b1e79cf125da14e5445f7d6331d397f003ad086d79656de5277640815930')})


Tran

#### Approach Etherscan Parsing

In [61]:
from bs4 import BeautifulSoup
import re

In [59]:
def parse_etherscan_for_transaction(transaction_hash):
    
    base_url = 'https://etherscan.io/tx/'
    url = base_url + transaction_hash
    
    # To Beat 403-Error
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    # Fetch 
    response = requests.get(url, headers=headers)    
    
    has_gas_token_flag = False

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        html_text = BeautifulSoup(response.content, 'html.parser')
        
        # Convert the soup object to string
        html_string = str(html_text)
        
        # Perform regex search on the HTML string for each word
        for word in gas_token_addresses.keys():
            matches = re.findall(r'\b' + re.escape(word) + r'\b', html_string, flags=re.IGNORECASE)
            
            # Print the matches
            if matches:
                print(f"'{gas_token_addresses[word]}' : {word} found!")
                has_gas_token_flag = True
    
    else:
        print("Could not Parse Etherscan")
        
    return has_gas_token_flag

In [62]:
transaction_with_chi_gastoken = "0x10504c0666a22466911d038d2b1c895b7ea5ec05cef107e46d3cae0121cfcda7"
parse_etherscan_for_transaction(transaction_with_chi_gastoken)

'CHI' : 0x0000000000004946c0e9F43F4Dee607b0eF1fA1c found!


True

In [67]:
def is_transaction_using_gas_token(block_number, address, web3):
    
    block = web3.eth.get_block(block_number, full_transactions=True)
    
    for transaction in block.transactions:  
        
        if not (transaction["from"] == address or transaction["to"] == address):
            continue
        else:
            hash = transaction["hash"].hex()
            print(f"Transaction: {hash}\n")
            parse_etherscan_for_transaction(hash)
            print("\n")
            break

In [79]:
counter = 1
for index, entry in df_insertion.sample(10).iterrows():
    block_nr = entry["Block Number"]
    address = entry["First Attacker"]
    print(address, counter)
    counter += 1
    is_transaction_using_gas_token(block_nr, address, web3)

0xdD07249e403979Bd79848c27aa5454C7E66BdEe7 1
Transaction: 0xb5ca994368371b87f3ea28c5a5e835d529ad03801a41c2caf1e9401af5be1aea



0x0242333254a79D0f0c8dfc1Cb06d9EBAD2746e83 2
Transaction: 0x807593b76750f967e25d42f9a4527c11cc581433202f65399a358cde848474b6



0x17ab75b3BD6B9872Fe8d021d6d72bab32f91aAEf 3
Transaction: 0x3f399733b6e72d3568a38152133e5764f257cc4d53a4774be28625edef707eae



0x86254Cb5A96C161E503d3255d67Fdddec056FEfe 4
Transaction: 0xcd527d23830e933ffcbf64b06c7c88eeb8f2b256454c33352df8ff535bbb3fcc



0xa21caEbD27a296678176aC886735bfd18F875B8f 5
Transaction: 0xc1ce101fd0d5a620462c35ceb29ee42ca1e47b1d74995c21db31f40662908788



0xB287Cb907fE97fB4116877a924B3dfd981b5ad7D 6
Transaction: 0x1a0e15d87d74528a03ea4df8ab76bcd45f5ce0cd06292d5ab9c4fec61444063b



0xfAE8A422969C6873363dae801B4EE1cAc2f2a687 7
Transaction: 0x1c7614af5a1c055e098fbfe54011d97c38abd091cc567ff271a60dd03040cf72

'GST2' : 0x0000000000b3F879cb30FE243b4Dfee438691c04 found!


0x401bF19cd1BA05650F0440c3d0c24FA3583854DD 8


#### Approach Etherscan API - Loop through internal transactions

In [82]:
import requests

def get_internal_transactions(tx_hash):
    # API endpoint
    url = 'https://api.etherscan.io/api'

    # Parameters
    params = {
        'module': 'account',
        'action': 'txlistinternal',
        'txhash': tx_hash,
        'apikey': '1PN1111XBM2W5HIQCSMQH6RA65JVYPQM1R'
    }

    try:
        # Sending GET request
        response = requests.get(url, params=params, timeout=3)
    
        # Checking if request was successful
        if response.status_code == 200:
            data = response.json()
            return data["result"]
        else:
            #print('Error occurred:', response.status_code)
            return None
        
    except requests.exceptions.Timeout:
        #print('Request did not go through: timeout occurred')
        return None

In [83]:
def is_gas_token_contract_in_internal_transaction(transaction_hash):
    
    gas_token_addresses = {"0x0000000000b3f879cb30fe243b4dfee438691c04": "GST2",
                           "0x88d60255f917e3eb94eae199d827dad837fac4cb": "GST1",
                           "0x0000000000004946c0e9f43f4dee607b0ef1fa1c": "CHI"}
    
    internal_transactions = get_internal_transactions(transaction_hash)
    
    if not internal_transactions:
        return False
    
    for transaction in internal_transactions:
        if transaction["from"] in gas_token_addresses.keys():
            #print(f"'{gas_token_addresses[transaction['from']]}' is used in {transaction['type']}")
            return True
        if transaction["to"] in gas_token_addresses.keys():
            #print(f"'{gas_token_addresses[transaction['to']]}' is used in {transaction['type']}")
            return True
    
    return False
    

In [84]:
def is_transaction_using_gas_token(block_number, address, web3):
    
    block = web3.eth.get_block(block_number, full_transactions=True)
    
    for transaction in block.transactions:  
        
        if not (transaction["from"] == address or transaction["to"] == address):
            continue
            
        else:
            transaction_hash = transaction["hash"].hex()
            #print(f"Transaction: {transaction_hash}\n")
            return is_gas_token_contract_in_internal_transaction(transaction_hash)

##### Override usedGasToken in already sampled data

In [92]:
def get_gas_token(file_name):
    
    df = pd.read_csv(f'../data/{file_name}', delimiter=',')

    entries = []
    
    for index, entry in tqdm(df.iterrows()):
        block_nr = entry["blockNumber"]
        address = entry["address"]
        entry["usedGasToken"] = is_transaction_using_gas_token(block_nr, address, web3)
        #print(address, counter)
        #counter += 1
        entries.append(entry)
    
    return pd.DataFrame(entries)

In [93]:
insertion_first_atk__with_gas_token_df = get_gas_token("insertion_atks_first_atk_2500.csv")
insertion_first_atk__with_gas_token_df.to_csv('../data/insertion_atks_first_atk_with_gas_token.csv')

2500it [47:42,  1.14s/it]


In [94]:
insertion_second_atk__with_gas_token_df = get_gas_token("insertion_atks_second_atk_2500.csv")
insertion_second_atk__with_gas_token_df.to_csv('../data/insertion_atks_second_atk_with_gas_token.csv')

2500it [45:35,  1.09s/it]


In [95]:
random_sampled_transactions_with_gas_token_df = get_gas_token("random_sampled_transactions.csv")
random_sampled_transactions_with_gas_token_df.to_csv('../data/random_sampled_transactions_with_gas_token.csv')

4821it [1:28:40,  1.10s/it]


Distribution of usedGasToken in attack data vs. randomly sampled data

In [103]:
df_first_atks = pd.read_csv(f'../data/insertion_atks_first_atk_with_gas_token.csv', delimiter=',')
df_second_atks = pd.read_csv(f'../data/insertion_atks_second_atk_with_gas_token.csv', delimiter=',')
df_first_atks.usedGasToken.value_counts() + df_second_atks.usedGasToken.value_counts()


usedGasToken
False    4112
True      888
Name: count, dtype: int64

In [104]:
df_random = pd.read_csv(f'../data/random_sampled_transactions_with_gas_token.csv', delimiter=',')
df_random.usedGasToken.value_counts()

usedGasToken
False    4809
True       12
Name: count, dtype: int64